# Import Thienpont_2018/lambrechts data:

In [5]:
import scanpy as sc
import sys
sys.path.append("../../../scripts/")
import pandas as pd
import numpy as np
import LCA_file_reading
import preprocessing
import reference_based_harmonizing

In [2]:
project_dir = "." # change to raw data dir HLCA core
target_adata = LCA_file_reading.read_file_Thienpont_2018(project_dir)

In [3]:
n_genes_before = target_adata.shape[1]
sc.pp.filter_genes(target_adata, min_counts=1)
# drop annotation label that is automatically created:
target_adata.var.drop("n_counts", axis=1, inplace=True)
n_genes_after = target_adata.shape[1]
print("number of genes removed:", n_genes_before - n_genes_after)

number of genes removed: 5716


In [6]:
# add up genes with duplicate names
target_adata = preprocessing.add_up_duplicate_gene_name_columns(target_adata)
print(target_adata.shape, "\n")
# correct gene names
renamer_dict = preprocessing.get_gene_renamer_dict(target_adata.var.index.tolist().copy())
n_genes_to_rename = np.sum(old != new for old, new in renamer_dict.items())
print(n_genes_to_rename, "GENES TO BE RENAMED (old, new):\n")
for old, new in renamer_dict.items():
    if len(new) > 0:
        print(old, new)
target_adata.var["original_gene_names"] = target_adata.var.index.tolist().copy()
translation_dict = dict(zip(target_adata.var.index, target_adata.var.index))
for gene_to_rename, new_name in renamer_dict.items():
    translation_dict[gene_to_rename] = new_name
target_adata.var.index = target_adata.var.index.map(translation_dict)
# add up duplicate genes again
# they might have emerged due to gene re-naming above
target_adata = preprocessing.add_up_duplicate_gene_name_columns(target_adata)

TO DO: STORE ENSEMBL IDS OF MERGED IDS AND MATCHING COUNTS IN A SEPARATE COLUMN!!!
Number of duplicate genes: 0
Index([], dtype='object')
(93575, 27978) 



/home/icb/lisa.sikkema/miniconda3/envs/HLCA_basic/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  


20 GENES TO BE RENAMED (old, new):

RGS5.1 RGS5
ATRIP.1 ATRIP
CYB561D2.1 CYB561D2
CFAP99.1 CFAP99
MATR3.1 MATR3
BLOC1S5.1 BLOC1S5
PRICKLE4.1 PRICKLE4
RAET1E-AS1.1 RAET1E-AS1
PGM5-AS1.1 PGM5-AS1
TIMM10B.1 TIMM10B
CCDC7.1 CCDC7
LINC00864.1 LINC00864
LINC01481.1 LINC01481
SPATA13.1 SPATA13
SERPINA3.1 SERPINA3
IGHV2-70.1 IGHV2-70
TMEM256-PLSCR3.1 TMEM256-PLSCR3
TBC1D26.1 TBC1D26
LINC01297.1 LINC01297
LINC01422.1 LINC01422
TO DO: STORE ENSEMBL IDS OF MERGED IDS AND MATCHING COUNTS IN A SEPARATE COLUMN!!!
Number of duplicate genes: 20
Index(['RGS5', 'ATRIP', 'CYB561D2', 'CFAP99', 'MATR3', 'BLOC1S5', 'PRICKLE4',
       'RAET1E-AS1', 'PGM5-AS1', 'TIMM10B', 'CCDC7', 'LINC00864', 'LINC01481',
       'SPATA13', 'SERPINA3', 'IGHV2-70', 'TMEM256-PLSCR3', 'TBC1D26',
       'LINC01297', 'LINC01422'],
      dtype='object')


/home/icb/lisa.sikkema/miniconda3/envs/HLCA_basic/lib/python3.7/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [7]:
harmonizing_df = reference_based_harmonizing.load_harmonizing_table(
    "../../../supporting_files/metadata_harmonization/HLCA_cell_type_reference_mapping_20220722.csv"
)
consensus_df = reference_based_harmonizing.create_consensus_table(harmonizing_df)
celltype_translation_df = (
    reference_based_harmonizing.create_orig_ann_to_consensus_translation_df(
        target_adata, consensus_df, harmonizing_df, verbose=False
    )
)

In [8]:
target_adata = reference_based_harmonizing.consensus_annotate_anndata(
    target_adata, celltype_translation_df, verbose=True
)

In [9]:
target_adata.X[:30,:10].toarray()

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 

In [10]:
target_adata

AnnData object with n_obs × n_vars = 93575 × 27958
    obs: 'n_genes_detected', 'total_counts', 'cell_from_tumor', 'subject_ID', 'tumor_site', 'original_celltype_ann', 'sample', 'study', 'study_long', 'dataset', 'last_author_PI', 'lung_vs_nasal', 'original_ann_level_1', 'original_ann_level_2', 'original_ann_level_3', 'original_ann_level_4', 'original_ann_level_5', 'original_ann_highest_res', 'original_ann_new'
    var: 'original_gene_names'

#### store full raw object:

In [11]:
target_adata.write("../../../data/HLCA_extended/extension_datasets/raw/Lambrechts/lambrechts.h5ad")
target_adata.write("../../../data/HLCA_extended/extension_datasets/ready/full/lambrechts.h5ad")


#### store subsetted object for scArches mapping (based on 2000 HLCA hvgs):

In [13]:
genes_to_keep = pd.read_csv("./genes_for_mapping.csv")

In [14]:
target_adata.var.head(5)

,original_gene_names
RP11-34P13.3,RP11-34P13.3
RP11-34P13.7,RP11-34P13.7
RP11-34P13.8,RP11-34P13.8
RP11-34P13.9,RP11-34P13.9
FO538757.3,FO538757.3


In [17]:
adata_subset = preprocessing.subset_and_pad_adata(genes_to_keep, target_adata)

not all genes were recovered, filling in 0 counts for 4 missing genes...


/home/icb/lisa.sikkema/miniconda3/envs/scRNAseq_analysis/lib/python3.7/site-packages/anndata/_core/merge.py:894: UserWarning: Only some AnnData objects have `.raw` attribute, not concatenating `.raw` attributes.
  UserWarning,


In [18]:
adata_subset.write("../../../data/HLCA_extended/extension_datasets/ready/subsetted/lambrechts_sub.h5ad")

... storing 'original_gene_names' as categorical
... storing 'gene_symbols' as categorical
